In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

C:\Users\AI\Anaconda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
X = train.iloc[:,2:]
y = train.iloc[:,1]

In [3]:
train

,ID,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,0,9 Crumlin Way,Ridgewood,520000,5,2,4.0,674,266,2007.0,35500,Butler Station,3000,09-2015\r,6030,-31.656206,115.720566,IRENE MCCORMACK CATHOLIC COLLEGE,1.079230,120.0
1,1,37 Sissinghurst Crescent,Landsdale,575000,5,4,2.0,540,301,2014.0,16900,Whitfords Station,8100,07-2018\r,6065,-31.800802,115.867675,LANDSDALE CHRISTIAN SCHOOL,0.667585,NaN
2,2,24 Tees Court,Mindarie,760000,4,2,2.0,781,220,1992.0,33500,Clarkson Station,3500,02-2020\r,6030,-31.682040,115.702600,MINDARIE SENIOR COLLEGE,0.650761,111.0
3,3,31 Brentham Street,Mount Hawthorn,1230000,2,1,2.0,425,185,1938.0,3500,Leederville Station,1500,07-2020\r,6016,-31.926046,115.838181,ARANMORE CATHOLIC COLLEGE,0.484465,106.0
4,4,46/1 Corkhill Street,North Fremantle,2010000,3,2,2.0,546,186,2004.0,12900,North Fremantle Station,800,10-2017\r,6159,-32.032079,115.759626,JOHN CURTIN COLLEGE OF THE ARTS,1.869050,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18505,18505,36 Kentucky Boulevard,Hocking,475000,4,2,2.0,533,198,2007.0,21200,Edgewater Station,4400,04-2018\r,6065,-31.764187,115.824151,WANNEROO SECONDARY COLLEGE,1.104533,131.0
18506,18506,10 Regatta Boulevard,Champion Lakes,485000,3,2,2.0,344,237,2010.0,22100,Kelmscott Station,1900,06-2011\r,6111,-32.114990,115.992834,JOHN WOLLASTON ANGLICAN COMMUNITY SCHOOL,1.816768,40.0
18507,18507,145 Coolgardie Avenue,Redcliffe,642000,3,1,1.0,1016,110,1953.0,8900,Bassendean Station,3100,01-2018\r,6104,-31.931320,115.950060,CYRIL JACKSON SENIOR CAMPUS,2.169696,NaN
18508,18508,12 Mckenzie Road,Samson,491000,4,2,2.0,718,176,1988.0,14600,Murdoch Station,5400,03-2009\r,6163,-32.070861,115.793834,SETON CATHOLIC COLLEGE,0.208669,91.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18510 entries, 0 to 18509
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                18510 non-null  int64  
 1   ADDRESS           18510 non-null  object 
 2   SUBURB            18510 non-null  object 
 3   PRICE             18510 non-null  int64  
 4   BEDROOMS          18510 non-null  int64  
 5   BATHROOMS         18510 non-null  int64  
 6   GARAGE            17143 non-null  float64
 7   LAND_AREA         18510 non-null  int64  
 8   FLOOR_AREA        18510 non-null  int64  
 9   BUILD_YEAR        16782 non-null  float64
 10  CBD_DIST          18510 non-null  int64  
 11  NEAREST_STN       18510 non-null  object 
 12  NEAREST_STN_DIST  18510 non-null  int64  
 13  DATE_SOLD         18510 non-null  object 
 14  POSTCODE          18510 non-null  int64  
 15  LATITUDE          18510 non-null  float64
 16  LONGITUDE         18510 non-null  float6

In [5]:
train["GARAGE"].value_counts()

2.0     11378
1.0      2906
3.0      1153
4.0      1096
6.0       246
5.0       178
8.0        63
7.0        54
12.0       16
10.0       12
9.0        10
14.0        7
13.0        7
11.0        6
16.0        3
20.0        2
17.0        1
18.0        1
32.0        1
21.0        1
26.0        1
99.0        1
Name: GARAGE, dtype: int64

In [6]:
train["GARAGE"] = train["GARAGE"].fillna(2.0)

In [7]:
train["BUILD_YEAR"].value_counts()

2006.0    522
2000.0    511
2002.0    498
2007.0    489
2004.0    487
         ... 
1889.0      1
1892.0      1
1907.0      1
1897.0      1
1941.0      1
Name: BUILD_YEAR, Length: 120, dtype: int64

In [8]:
train["BUILD_YEAR"] = train["BUILD_YEAR"].fillna(2006.0)

In [9]:
train["NEAREST_SCH_RANK"].value_counts()

53.0     414
92.0     386
129.0    380
131.0    306
35.0     300
        ... 
71.0      13
55.0      13
88.0      10
5.0        3
15.0       2
Name: NEAREST_SCH_RANK, Length: 103, dtype: int64

In [10]:
train["NEAREST_SCH_RANK"] = train["NEAREST_SCH_RANK"].fillna(53.0)

In [25]:
train.isnull().sum()

ADDRESS             0
BATHROOMS           0
BEDROOMS            0
BUILD_YEAR          0
CBD_DIST            0
DATE_SOLD           0
FLOOR_AREA          0
GARAGE              0
ID                  0
LAND_AREA           0
LATITUDE            0
LONGITUDE           0
NEAREST_SCH         0
NEAREST_SCH_DIST    0
NEAREST_SCH_RANK    0
NEAREST_STN         0
NEAREST_STN_DIST    0
POSTCODE            0
PRICE               0
SUBURB              0
dtype: int64

In [12]:
test["GARAGE"] = test["GARAGE"].fillna(2.0)
test["BUILD_YEAR"] = test["BUILD_YEAR"].fillna(2006.0)
test["NEAREST_SCH_RANK"] = test["NEAREST_SCH_RANK"].fillna(53.0)

In [24]:
test.isnull().sum()

ADDRESS             0
BATHROOMS           0
BEDROOMS            0
BUILD_YEAR          0
CBD_DIST            0
DATE_SOLD           0
FLOOR_AREA          0
GARAGE              0
ID                  0
LAND_AREA           0
LATITUDE            0
LONGITUDE           0
NEAREST_SCH         0
NEAREST_SCH_DIST    0
NEAREST_SCH_RANK    0
NEAREST_STN         0
NEAREST_STN_DIST    0
POSTCODE            0
SUBURB              0
dtype: int64

In [14]:
X

,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,Ridgewood,520000,5,2,4.0,674,266,2007.0,35500,Butler Station,3000,09-2015\r,6030,-31.656206,115.720566,IRENE MCCORMACK CATHOLIC COLLEGE,1.079230,120.0
1,Landsdale,575000,5,4,2.0,540,301,2014.0,16900,Whitfords Station,8100,07-2018\r,6065,-31.800802,115.867675,LANDSDALE CHRISTIAN SCHOOL,0.667585,NaN
2,Mindarie,760000,4,2,2.0,781,220,1992.0,33500,Clarkson Station,3500,02-2020\r,6030,-31.682040,115.702600,MINDARIE SENIOR COLLEGE,0.650761,111.0
3,Mount Hawthorn,1230000,2,1,2.0,425,185,1938.0,3500,Leederville Station,1500,07-2020\r,6016,-31.926046,115.838181,ARANMORE CATHOLIC COLLEGE,0.484465,106.0
4,North Fremantle,2010000,3,2,2.0,546,186,2004.0,12900,North Fremantle Station,800,10-2017\r,6159,-32.032079,115.759626,JOHN CURTIN COLLEGE OF THE ARTS,1.869050,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18505,Hocking,475000,4,2,2.0,533,198,2007.0,21200,Edgewater Station,4400,04-2018\r,6065,-31.764187,115.824151,WANNEROO SECONDARY COLLEGE,1.104533,131.0
18506,Champion Lakes,485000,3,2,2.0,344,237,2010.0,22100,Kelmscott Station,1900,06-2011\r,6111,-32.114990,115.992834,JOHN WOLLASTON ANGLICAN COMMUNITY SCHOOL,1.816768,40.0
18507,Redcliffe,642000,3,1,1.0,1016,110,1953.0,8900,Bassendean Station,3100,01-2018\r,6104,-31.931320,115.950060,CYRIL JACKSON SENIOR CAMPUS,2.169696,NaN
18508,Samson,491000,4,2,2.0,718,176,1988.0,14600,Murdoch Station,5400,03-2009\r,6163,-32.070861,115.793834,SETON CATHOLIC COLLEGE,0.208669,91.0


In [15]:
train.shape, test.shape

((18510, 20), (15146, 19))

In [16]:
set(train.columns) - set(test.columns)

{'PRICE'}

In [17]:
train.sort_index(axis=1, inplace=True)
test.sort_index(axis=1, inplace=True)

In [18]:
train.columns

Index(['ADDRESS', 'BATHROOMS', 'BEDROOMS', 'BUILD_YEAR', 'CBD_DIST',
       'DATE_SOLD', 'FLOOR_AREA', 'GARAGE', 'ID', 'LAND_AREA', 'LATITUDE',
       'LONGITUDE', 'NEAREST_SCH', 'NEAREST_SCH_DIST', 'NEAREST_SCH_RANK',
       'NEAREST_STN', 'NEAREST_STN_DIST', 'POSTCODE', 'PRICE', 'SUBURB'],
      dtype='object')

In [19]:
test.columns

Index(['ADDRESS', 'BATHROOMS', 'BEDROOMS', 'BUILD_YEAR', 'CBD_DIST',
       'DATE_SOLD', 'FLOOR_AREA', 'GARAGE', 'ID', 'LAND_AREA', 'LATITUDE',
       'LONGITUDE', 'NEAREST_SCH', 'NEAREST_SCH_DIST', 'NEAREST_SCH_RANK',
       'NEAREST_STN', 'NEAREST_STN_DIST', 'POSTCODE', 'SUBURB'],
      dtype='object')

In [20]:
from sklearn.model_selection import train_test_split   # 트레인과 테스트를 알아서 나눠죠
X_train, X_test, y_train, y_test = train_test_split(X,               #독립 변수 
                                                    y,               #종속 변수
                                                    test_size=0.4,   #검증 30%   => 트레인 사이트는 70으 된다.
                                                    random_state=555)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=555)

In [28]:
X_train = train.drop(['ADDRESS','SUBURB','DATE_SOLD','NEAREST_STN','NEAREST_SCH','PRICE'], axis=1)
y_train = train.PRICE
X_test = test.drop(['ADDRESS','SUBURB','DATE_SOLD','NEAREST_STN','NEAREST_SCH'], axis=1)
X_val = X_val.drop(['SUBURB','DATE_SOLD','NEAREST_STN','NEAREST_SCH'], axis=1)

In [33]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3702 entries, 261 to 11012
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PRICE             3702 non-null   int64  
 1   BEDROOMS          3702 non-null   int64  
 2   BATHROOMS         3702 non-null   int64  
 3   GARAGE            3417 non-null   float64
 4   LAND_AREA         3702 non-null   int64  
 5   FLOOR_AREA        3702 non-null   int64  
 6   BUILD_YEAR        3372 non-null   float64
 7   CBD_DIST          3702 non-null   int64  
 8   NEAREST_STN_DIST  3702 non-null   int64  
 9   POSTCODE          3702 non-null   int64  
 10  LATITUDE          3702 non-null   float64
 11  LONGITUDE         3702 non-null   float64
 12  NEAREST_SCH_DIST  3702 non-null   float64
 13  NEAREST_SCH_RANK  2487 non-null   float64
dtypes: float64(6), int64(8)
memory usage: 433.8 KB


In [34]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test)

C:\Users\AI\Anaconda3\lib\site-packages\xgboost\data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [35]:
params = {'max_depth' : 3,
         'eta' : 0.1, 
         'objective' : 'binary:logistic',
         'eval_metric' : 'logloss',
         'early_stoppings' : 100 }

In [36]:
xgb_model = xgb.train(params = params, dtrain = dtrain, num_boost_round = 400, 
                        early_stopping_rounds = 100, evals=[(dtrain,'train'),(dtest,'test')])

[08:34:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "early_stoppings" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBoostError: [08:34:44] C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:103: label must be in [0,1] for logistic regression

In [ ]:
# 예측하기, 확률값으로 반환됨
y_pred_probs = xgb_model.predict(dtest)

# 0또는 1로 변경
y_preds = [1 if x>0.5 else 0 for x in y_pred_probs]

#성능 평가
print(confusion_matrix(y_test, y_preds))
print(classification_report(y_test, y_preds))
print(roc_auc_score(y_test, y_preds)

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(xgb_model, ax=ax)

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
xgb.fit(X_train, y_train)
y_preds = xgb.predict(X_test)

#성능 평가
print(confusion_matrix(y_test, y_preds))
print(classification_report(y_test, y_preds))
print(roc_auc_score(y_test, y_preds))